In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import json
from openai import OpenAI

In [10]:
# Read list from a JSON file
with open('data/department_list.json', 'r') as file:
    department_list = json.load(file)

In [11]:
# Base URL for course descriptions
BASE_URL = "https://vancouver.calendar.ubc.ca/course-descriptions/subject/"

# Function to scrape courses for a department
def scrape_department_courses(department_code):
    url = f"{BASE_URL}{department_code}v"
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Failed to fetch data for {department_code}: {response.status_code}")
        return []

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract all course blocks (using updated selectors based on the provided HTML structure)
    courses = []
    for course in soup.find_all("article", class_="node node--type-course node--promoted node--view-mode-teaser"):
        # Extract course code, name, and credit hours from the h3 tag
        header = course.find("h3")
        if header:
            course_details = header.text.strip()
            # Regex to match the course code, credit, and course name
            match = re.match(r"(?P<code>[A-Z]+_V \d+) \((?P<credits>\d+)\)\s+(?P<name>.+)$", course_details)
            if match:
                course_code = match.group('code')
                course_name = match.group('name')
                credits = int(match.group('credits'))
            else:
                continue
        
        # Extract course description and prerequisites from the p tag
        description_block = course.find("p")
        if description_block:
            description_text = description_block.text.strip()
            # Regex to extract prerequisites text
            prereq_match = re.search(r"Prerequisite[s]*: (.+?)(?:\s[Cc]orequisite[s]*:|$)", description_text)
            prerequisites = prereq_match.group(1) if prereq_match else None
            coreq_match = re.search(r"Corequisite[s]*: (.+)", description_text, re.IGNORECASE)
            corequisites = coreq_match.group(1) if coreq_match else None

            # Append course data
            courses.append({
                "course_code": course_code,
                "course_name": course_name,
                "credits": credits,
                "description": description_text,
                "prerequisites": prerequisites,
                "corequisites": corequisites
            })

    return courses

# Scrape multiple departments
departments = department_list  # Add more department codes as needed(!!!, replace the full course list after finish future design)
all_courses = []
for dept in departments:
    print(f"Scraping {dept}...")
    courses = scrape_department_courses(dept)
    all_courses.extend(courses)

Scraping ACAM...
Scraping ADHE...
Scraping AFST...
Scraping AMNE...
Scraping ANAT...
Scraping ANTH...
Scraping APBI...
Scraping APSC...
Scraping ARBC...
Scraping ARBM...
Scraping ARCH...
Scraping ARCL...
Scraping ARTC...
Failed to fetch data for ARTC: 429
Scraping ARTH...
Failed to fetch data for ARTH: 429
Scraping ARTS...
Failed to fetch data for ARTS: 429
Scraping ASIA...
Failed to fetch data for ASIA: 429
Scraping ASIC...
Failed to fetch data for ASIC: 429
Scraping ASL...
Failed to fetch data for ASL: 429
Scraping ASLA...
Failed to fetch data for ASLA: 429
Scraping ASTR...
Failed to fetch data for ASTR: 429
Scraping ASTU...
Failed to fetch data for ASTU: 429
Scraping ATSC...
Failed to fetch data for ATSC: 429
Scraping AUDI...
Failed to fetch data for AUDI: 429
Scraping BEST...
Failed to fetch data for BEST: 429
Scraping BIOC...
Failed to fetch data for BIOC: 429
Scraping BIOL...
Failed to fetch data for BIOL: 429
Scraping BIOT...
Failed to fetch data for BIOT: 429
Scraping BMEG...
F

In [12]:
# Convert to DataFrame and save
df_courses = pd.DataFrame(all_courses)

In [13]:
df_courses['department'] = df_courses['course_code'].str.split().str[0]
df_courses['department'] = df_courses['department'].str.replace('_V', '')
df_courses['course_code'] = df_courses['course_code'].str.split().str[1].astype(int)

In [14]:
df_courses = df_courses[["department", "course_code", "course_name", "credits", "prerequisites", "corequisites", "description"]]

In [15]:
ubc_courses_df = df_courses

# Redefine the parsing function with handling for brackets as "all_of"
def parse_prerequisite(prerequisite_string):
    """
    Converts a human-readable prerequisite string into a Prerequisite object representation.
    """
    if not isinstance(prerequisite_string, str) or not prerequisite_string.strip():
        return None

    prerequisite_string = prerequisite_string.lower()

    # Handle simple course requirements (e.g., "CPSC 221")
    single_course_match = re.match(r"([a-z]+ \d+)", prerequisite_string, re.IGNORECASE)
    if single_course_match:
        return f'Prerequisite(type="course", details="{single_course_match.group(1).upper()}")'

    # Handle "one of" and "all of" scenarios
    if "one of" in prerequisite_string:
        courses = re.findall(r"([a-z]+ \d+)", prerequisite_string, re.IGNORECASE)
        parsed_courses = ", ".join([f'Prerequisite(type="course", details="{course.upper()}")' for course in courses])
        return f'Prerequisite(type="one_of", children=[{parsed_courses}])'

    if "all of" in prerequisite_string:
        courses = re.findall(r"([a-z]+ \d+)", prerequisite_string, re.IGNORECASE)
        parsed_courses = ", ".join([f'Prerequisite(type="course", details="{course.upper()}")' for course in courses])
        return f'Prerequisite(type="all_of", children=[{parsed_courses}])'

    # Handle brackets as "all_of" scenario (e.g., "[CPSC 220, CPSC 111]")
    bracket_match = re.findall(r"\[([^\]]+)\]", prerequisite_string)
    if bracket_match:
        courses = re.findall(r"([a-z]+ \d+)", bracket_match[0], re.IGNORECASE)
        parsed_courses = ", ".join([f'Prerequisite(type="course", details="{course.upper()}")' for course in courses])
        return f'Prerequisite(type="all_of", children=[{parsed_courses}])'

    # Handle credit requirements (e.g., "6 credits of MATH or STAT")
    credit_match = re.match(r"(\d+) credits of ([a-z]+( or [a-z]+)*)", prerequisite_string, re.IGNORECASE)
    if credit_match:
        credit_value = credit_match.group(1)
        departments = [dept.strip().upper() for dept in credit_match.group(2).split("or")]
        return f'Prerequisite(type="credits", details={{"min_credits": {credit_value}, "departments": {departments}}})'

    # Handle fallback (return original string as a comment for manual refinement)
    return f'/* Unparsed: {prerequisite_string} */'

# Apply the updated parsing logic
ubc_courses_df["prerequisites"] = ubc_courses_df["prerequisites"].apply(parse_prerequisite)
ubc_courses_df["corequisites"] = ubc_courses_df["corequisites"].apply(parse_prerequisite)

ubc_courses_df

C:\Users\David\AppData\Local\Temp\ipykernel_23088\639606157.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['parsed_prerequisites'] = sample_df['prerequisites'].apply(
C:\Users\David\AppData\Local\Temp\ipykernel_23088\639606157.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['parsed_corequisites'] = sample_df['corequisites'].apply(


,department,course_code,course_name,credits,prerequisites,corequisites,description,parsed_prerequisites,parsed_corequisites
0,ACAM,250,Asian Canadians in Popular Culture,3,None,None,Popular culture's role in the production of As...,[],[]
1,ACAM,300,Dis/Orienting Asian Canada,3,None,None,"The histories, cultures, social dynamics, and ...",[],[]
2,ACAM,310,Asian Canadian Cultural Studies,3,None,None,Examines Asian Canadian communities and cultur...,[],[]
3,ACAM,350,Asian Canadian Community-Based Media,3,None,None,Digital media production in the context of com...,[],[]
4,ADHE,327,Teaching Adults,3,None,None,"Planning, conducting and evaluating instructio...",[],[]
5,ADHE,328,Institutions of Adult Education,3,None,None,"The history, roles, and activities of institut...",[],[]
6,ADHE,329,"Developing Short Courses, Workshops and Seminars",3,None,None,Organization and administration of adult educa...,[],[]
7,ADHE,330,The Community Practice of Adult Education,3,None,None,Community based adult education with particula...,[],[]
8,ADHE,375,Diploma Seminar and Internship in Adult Education,6,None,None,[2-3-0],[],[]
9,ADHE,412,An Overview of Adult Education,3,None,None,Survey of adult education theory and practice ...,[],[]


In [16]:
df_courses.to_csv("data/ubc_courses.csv", index=False)
print("Course data saved to ubc_courses.csv.")

Course data saved to ubc_courses.csv.
